# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint

# Import API key
from api_keys import g_key

In [ ]:
### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_path = "../weatherData1_df.csv"
weather_details = pd.read_csv(weather_path)
weather_details

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,0,Hermanus,ZA,1614148381,-34.4187,19.2345,64.99,93,21,6.87
1,1,Coquimbo,CL,1614148382,-29.9533,-71.3436,59.00,94,90,3.44
2,2,Key West,US,1614148383,24.5557,-81.7826,73.00,84,0,3.00
3,3,Busselton,AU,1614148383,-33.6500,115.3333,82.00,53,76,4.00
4,4,Kahului,US,1614148384,20.8947,-156.4700,73.40,68,20,16.11
...,...,...,...,...,...,...,...,...,...,...
594,594,Calabozo,VE,1614148558,8.9242,-67.4293,76.35,73,100,9.17
595,595,Kirkenes,NO,1614148558,69.7271,30.0458,21.20,85,20,12.66
596,596,Iracoubo,GF,1614148558,5.4802,-53.2011,77.25,87,100,14.25
597,597,Marmande,FR,1614148559,44.5000,0.1667,53.01,93,4,7.00


In [3]:
Location = weather_details[['Latitude', 'Longitude']] # calling out the Lat and Long
weight_humidity = weather_details['Humidity'].astype(float) # humidity as weight 

In [5]:
gmaps.configure(api_key=g_key)

# Customize the size of the figure



fig = gmaps.figure()

heatmap = gmaps.heatmap_layer(Location, weights=weight_humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)
fig.add_layer(heatmap)


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
Fit_weather = weather_details.loc[(weather_details["Max Temperature"] > 70) & (weather_details["Max Temperature"] < 80) & (weather_details["Cloudiness"] == 0), :]
Fit_weather_df = Fit_weather.dropna(how='any')
Fit_weather_df .reset_index(inplace=True)
del Fit_weather_df ['index']
Fit_weather_df .head()


,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,2,Key West,US,1614148383,24.5557,-81.7826,73.00,84,0,3.00
1,38,Hobart,AU,1614148400,-42.8794,147.3294,72.00,39,0,11.50
2,44,São Filipe,CV,1614148405,14.8961,-24.4956,71.29,70,0,6.49
3,57,Acapulco de Juárez,MX,1614148219,16.8634,-99.8901,73.60,69,0,3.96
4,86,Quang Ngai,VN,1614148287,15.1167,108.8000,77.18,68,0,6.73


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = pd.DataFrame(Fit_weather_df, columns=["City", "Country", "Latitude", "Longitude"])
hotel_df["Hotel Name"] = " "
hotel_df 

,City,Country,Latitude,Longitude,Hotel Name
0,Key West,US,24.5557,-81.7826,
1,Hobart,AU,-42.8794,147.3294,
2,São Filipe,CV,14.8961,-24.4956,
3,Acapulco de Juárez,MX,16.8634,-99.8901,
4,Quang Ngai,VN,15.1167,108.8000,
5,Biltine,TD,14.5333,20.9167,
6,Morón,AR,-34.6534,-58.6198,
7,Atuona,PF,-9.8000,-139.0333,
8,São João da Barra,BR,-21.6403,-41.0511,
9,Salalah,OM,17.0151,54.0924,


In [8]:
#To set paprameters to search for hotels within 5000meters, the radius must be set to 5000
search = f"{hotel_df['Latitude'][0]},{hotel_df['Longitude'][0]}"
search_word = "hotel"
search_radius = 5000
search_type = "hotel"


# set parameters dictionary
params = {
    "location": search,
    "keyword": search_word,
    "radius": search_radius,
    "type": search_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)
response
reponse_json =response.json()
pprint.pprint(reponse_json, indent= 0.5)

{'html_attributions': [],
'next_page_token': 'ATtYBwJ0tFz9W3U50pbDYv1RtSrKqqnt0heUoiGHVkPSCJWs3VuGkgEENgPL04jkQo8xEo622aYazrIvADvQ5faRm1tOUWyySCRdQPF69kULUWIMCaYQOkgIqBlpePILV0IZHrXQDANIu814-2pqcnBbJV_ShsRnM3xmCZUzz0UQQyMvStjSgrFblVmLaZYaPX5GMv8UaD3xQ7VAEBbLWm5NszxnuwV6WlhZ_GkFta5uEan_XkAinHU0vAKHNOwZNcpYDtuTQMu-Tk6Ui16p5AGAOtg2h37TgDvJiSicM9MHJtCrh04ZwhNXjliAbUSXviZouxK1OocjJuLokolpa8FQSmy0JPywDk1UvaH07pPsxH3mwjhI4Aab9W9KIEw2mGMgTx0NFB-Pamk24BWHBG2KIRMSK6t5vwp8DIBCa1ojWEskDGBzucbWb_T6ZNs6Lto',
'results': [{'business_status': 'OPERATIONAL',
           'geometry': {'location': {'lat': 24.5680768,
                                   'lng': -81.77011639999999},
                       'viewport': {'northeast': {'lat': 24.56942142989272,
                                                'lng': -81.76885992010729},
                                   'southwest': {'lat': 24.56672177010728,
                                                'lng': -81.77155957989274}}},
           'icon': 'https://m

In [9]:
hotel_df.iloc[0][0]

# params = {
#     "location": search,
#     "keyword": search_word,
#     "radius": search_radius,
#     "type": search_type,
#     "key": g_key
# }

'Key West'

In [10]:
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
#Plot markers on top of the heatmap.

hotel_name = []

for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Latitude']
    lng = hotel_df.loc[i]['Longitude']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    json_format = requested.json()
    try:
        hotel_name.append(json_format['results'][0]['name'])
    except:
        hotel_name.append("")
        
hotel_df["Hotel Name"] = hotel_name
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()


,City,Country,Latitude,Longitude,Hotel Name
0,Key West,US,24.5557,-81.7826,Key West
1,Hobart,AU,-42.8794,147.3294,Hobart
2,São Filipe,CV,14.8961,-24.4956,Sao Filipe
3,Acapulco de Juárez,MX,16.8634,-99.8901,Acapulco
4,Quang Ngai,VN,15.1167,108.8000,Quảng Ngãi


In [35]:
hotel_name

['Key West',
 'Hobart',
 'Sao Filipe',
 'Acapulco',
 'Quảng Ngãi',
 'Biltine',
 'Morón',
 'Atuona',
 'Grussaí',
 'Salalah',
 'Tsiroanomandidy',
 'Manzanillo',
 'Sokone',
 'Road Town',
 'Port Alfred',
 'Dalbandin',
 'Doha',
 'Napier',
 'Comodoro Rivadavia',
 'Kharan',
 'Dolphin Coast',
 'Santarém',
 'Linguere',
 'Plettenberg Bay',
 'Liangshan Yi Autonomous Prefecture',
 'Martinópolis']

In [ ]:
#NAddress

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(Location)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))